In [2]:
# This is experimental code, that is why it didn't make it to the ecomplexity library yet. Still working on it...
import sys
import numpy as np
import pandas as pd
import networkx as nx
from collections import defaultdict
sys.path.append("../..")
from libraries import network_distance as nd

In [5]:
# Loading the input. We need the Product Spaces (both RCA and RPOP)
# We also need MCP (both exp and imp) and MCCP because we use also importer information
ps_rca = nx.read_edgelist("ps_nc_bb_0.01_rca.csv", delimiter = "\t", data = [("count", int), ("significance", float)], nodetype = str)
ps_rpop = nx.read_edgelist("ps_nc_bb_0.1_rpop.csv", delimiter = "\t", data = [("count", int), ("significance", float)], nodetype = str)
mcp_exp = pd.read_csv("../../data/processed/mcp_exp.csv", sep = "\t", dtype = {"prod": str})
mcp_imp = pd.read_csv("../../data/processed/mcp_imp.csv", sep = "\t", dtype = {"prod": str})
mccp = pd.read_csv("../../data/processed/mccp.csv", sep = "\t", dtype = {"prod": str})
mccp

exporter importer  prod     dollars          rca       rpop
0            ABW      ARE  2204    123032.0   226.737073   0.221308
1            ARG      ARE  2204   7513327.0     3.586126   2.440997
2            AUS      ARE  2204  47438249.0     3.641529  24.573280
3            AUT      ARE  2204    510334.0     0.085443   0.481668
4            BEL      ARE  2204    626244.0     0.021480   0.513251
...          ...      ...   ...         ...          ...        ...
4167049      NAM      COD  2612   1831608.0    83.570042  16.562123
4167050      MYS      LAO  2612     16625.0     4.573387   0.304554
4167051      IND      LKA  2612      3263.0     0.003446   0.001652
4167052      NAM      MWI  2612   1911837.0  2650.753516  70.211452
4167053      FRA      NCL  2612      1529.0     0.007957   0.015154

[4167054 rows x 6 columns]

In [16]:
# Let's set up some mock parameters that will make it easier to integrate this code into the library
variant = "rpop"
threshold = 1

In [17]:
if variant == "rca":
    # Depending on the variant we have a different order in the products (as they appear in the Product Space)
    prod_order = list(ps_rca.nodes)
    mcp_exp_rca = (pd.pivot_table(data = mcp_exp, index = "exporter", columns = "prod", values = variant).fillna(0.0) >= threshold).astype(int)[prod_order]
    mcp_imp_rca = (pd.pivot_table(data = mcp_imp, index = "importer", columns = "prod", values = variant).fillna(0.0) >= threshold).astype(int)[prod_order]
    mccp_rca = (pd.pivot_table(data = mccp, index = ["exporter", "importer"], columns = "prod", values = variant).fillna(0.0) >= threshold).astype(int)[prod_order]
    # Q is the key of the annihilation measure, and it might take a little bit to calculate
    # Since it is entirely dependent on the Product Space, we can cache it because it will be reused many times, saving us computation
    Q = nd._annihilation_Q(nx.adjacency_matrix(ps_rca).todense().astype(float))
elif variant == "rpop":
    prod_order = list(ps_rpop.nodes)
    mcp_exp_rca = (pd.pivot_table(data = mcp_exp, index = "exporter", columns = "prod", values = "rpop").fillna(0.0) >= threshold).astype(int)[prod_order]
    mcp_imp_rca = (pd.pivot_table(data = mcp_imp, index = "importer", columns = "prod", values = "rpop").fillna(0.0) >= threshold).astype(int)[prod_order]
    mccp_rca = (pd.pivot_table(data = mccp, index = ["exporter", "importer"], columns = "prod", values = "rpop").fillna(0.0) >= threshold).astype(int)[prod_order]
    Q = nd._annihilation_Q(nx.adjacency_matrix(ps_rpop).todense().astype(float))

Q

array([[ 1.06461641e+00+0.j,  3.57600790e-02+0.j,  6.51170978e-02+0.j,
        ..., -1.03028833e-02+0.j, -7.46898384e-03+0.j,
        -3.89362234e-03+0.j],
       [ 2.20520487e-01+0.j,  1.15784848e+00+0.j,  2.46711240e-02+0.j,
        ..., -6.79892795e-02+0.j, -5.33115579e-02+0.j,
        -1.50564192e-02+0.j],
       [ 2.82174090e-01+0.j,  1.73364655e-02+0.j,  1.23674159e+00+0.j,
        ..., -6.05737223e-02+0.j, -5.08952122e-02+0.j,
        -2.04955767e-02+0.j],
       ...,
       [-1.71714722e-03+0.j, -1.83754809e-03+0.j, -2.32975855e-03+0.j,
        ...,  4.04631404e+00+0.j, -8.04586007e-04+0.j,
        -2.80013639e-03+0.j],
       [-1.24483064e-03+0.j, -1.44085292e-03+0.j, -1.95750816e-03+0.j,
        ..., -8.04586007e-04+0.j,  2.81044926e+00+0.j,
        -2.23387071e-03+0.j],
       [-6.48937057e-04+0.j, -4.06930249e-04+0.j, -7.88291412e-04+0.j,
        ..., -2.80013639e-03+0.j, -2.23387071e-03+0.j,
         2.05731202e+00+0.j]])

In [18]:
# All complexities are initialized as vectors of ones
eci_exp = np.ones(mcp_exp_rca.shape[0])
eci_imp = np.ones(mcp_imp_rca.shape[0])
pci = np.ones(mcp_exp_rca.shape[1])
# A bunch of normalizations to make sure we're dealing with stochastic matrices
mcp_exp_rca = (mcp_exp_rca.T / mcp_exp_rca.sum(axis = 1)).T
mcp_imp_rca = (mcp_imp_rca.T / mcp_imp_rca.sum(axis = 1)).T
mccp_rca = (mccp_rca.T / mccp_rca.sum(axis = 1)).T
# MCCP is a 3D array: product, exporter, importer, so we need to deal with it in the proper shape
mccp_rca = np.nan_to_num(mccp_rca.to_xarray().to_array().data) 
mccp_rca.shape

(994, 164, 164)

In [19]:
# Annihilation is a node vector distance measure and thus it wants two vectors and a network
# It returns the distance between the vectors on the network
# One vector is the probability distribution of the export of a country
# The other vector must be our idea of what the "best" exporter could possibly be
# For now we say that the perfect exporter/importer is the one exporting/importing complex products to/from complex importers/exporters
# Note: this for now is silly, because eci_imp, eci_exp, and pci are all vectors of ones, but this code is embeddable in a loop to recursively update
# (Spoiler alert: I already put it in the loop and the loop makes the measure identical at best, but most likely worse...)
ideal_exp = (np.outer(eci_imp, pci) * mcp_imp_rca).sum()
ideal_imp = (np.outer(eci_exp, pci) * mcp_exp_rca).sum()
ideal_exp /= ideal_exp.sum()
ideal_imp /= ideal_imp.sum()

ideal_exp

prod
0102    0.000804
0104    0.000677
0201    0.000531
0202    0.001380
0807    0.000500
          ...   
8302    0.000804
5903    0.001295
7007    0.000588
8404    0.001454
8516    0.000842
Length: 994, dtype: float64

In [20]:
# Now we calculate all distances of exporters/importers to the ideal exporter/importer using the annihilation meaure
# The measure is described in Sec 3.1.3 here: https://www.michelecoscia.com/wp-content/uploads/2020/12/3416509.pdf
# Since we're calculating the distance to the ideal, the lower this complexity score the better
eci_exp_new = np.zeros(mcp_exp_rca.shape[0])
eci_imp_new = np.zeros(mcp_imp_rca.shape[0])

for i in range(len(eci_exp)):
    eci_exp_new[i] = nd._annihilation(mcp_exp_rca.iloc[i], ideal_exp, Q)

for i in range(len(eci_imp)):
    eci_imp_new[i] = nd._annihilation(mcp_imp_rca.iloc[i], ideal_imp, Q)

eci_exp_new

array([0.14638217, 0.45763341, 0.46924676, 0.21588653, 0.06173453,
       0.16340324, 0.23951753, 0.08417578, 0.06733613, 0.28715087,
       0.05532473, 0.4295765 , 0.59368909, 0.42992077, 0.08431585,
       0.119536  , 0.18755878, 0.12853399, 0.10896178, 0.22674212,
       0.23931143, 0.18674462, 0.20324839, 0.22782036, 0.0995126 ,
       0.09412371, 0.14187846, 0.10816901, 0.27591312, 0.39902544,
       0.43355931, 0.27644004, 0.19351493, 0.11825103, 0.33978981,
       0.12763738, 0.09332455, 0.07940048, 0.0629267 , 0.31009241,
       0.07883497, 0.17734583, 0.43612596, 0.20499851, 0.20267398,
       0.0508828 , 0.08390672, 0.44162421, 0.11306902, 0.2423519 ,
       0.07032417, 0.33626141, 0.09064248, 0.21472868, 0.31004256,
       0.18036885, 0.50382249, 0.41554195, 0.10963994, 0.20344322,
       0.2319657 , 0.09075858, 0.23596828, 0.09267403, 0.48179171,
       0.09297463, 0.19517598, 0.30816829, 0.10377418, 0.23524395,
       0.50713766, 0.14588711, 0.11240152, 0.07418195, 0.28059

In [21]:
# The product complexity is simply the weighted average complexity of the trade links through which it is exchanged
# The eci_exp (X) eci_imp (outer product) is an exporter, importer matrix, with the product of exp_i imp_j in its ij-th cell
trade_link_complexity = np.outer(eci_exp_new, eci_imp_new)
# It can be thus elementwise multiplied with MCCP, which then is collapsed by summing the matrices referring to each product
pci_new = (trade_link_complexity * mccp_rca).sum(axis = 1).sum(axis = 1)
pci_new

array([0.64500654, 0.65764587, 0.24549821, 1.03156619, 0.70803821,
       0.70818127, 1.63209811, 1.99303693, 0.59686064, 0.2032619 ,
       0.28025048, 0.37598862, 0.44140679, 0.87817979, 0.47081119,
       0.20638959, 0.13340544, 0.1594497 , 1.18956891, 0.58960229,
       1.00069485, 0.33262153, 0.27186479, 1.11685705, 5.22147457,
       0.54358229, 0.2357063 , 1.27333804, 2.3111422 , 0.58898936,
       1.37101742, 0.25734475, 0.65145185, 0.36882732, 1.19587902,
       2.26816066, 0.39994605, 0.31970811, 0.48263712, 0.35230442,
       1.23062113, 0.31811636, 0.66442373, 1.56912279, 0.5399387 ,
       0.84089548, 1.5245228 , 1.25434156, 0.47193305, 0.18396249,
       0.7063033 , 0.10659238, 0.08041849, 0.59541096, 2.48136227,
       0.06790222, 0.39494756, 0.38108008, 0.43736975, 0.22629732,
       0.20909557, 0.3251742 , 0.28234409, 0.02683714, 0.27845364,
       0.78628563, 1.50994254, 0.5204062 , 0.83427375, 0.49919766,
       0.12891237, 0.38783248, 0.13353015, 0.1225564 , 0.10737

In [22]:
# We normalize everything so that a value larger than 1 equals a complexity value above the average
# Remember that lower is better!
eci_exp_new /= eci_exp_new.mean()
eci_imp_new /= eci_imp_new.mean()
pci_new /= pci_new.mean()

eci_exp_new

array([0.65455934, 2.04634363, 2.09827364, 0.96535353, 0.27605081,
       0.73067037, 1.0710214 , 0.3763986 , 0.30109877, 1.28401761,
       0.24738889, 1.92088496, 2.65472728, 1.92242439, 0.37702492,
       0.53451461, 0.8386838 , 0.5747498 , 0.48723114, 1.01389517,
       1.07009979, 0.83504319, 0.90884111, 1.0187166 , 0.44497838,
       0.42088153, 0.63442064, 0.48368619, 1.23376712, 1.78427353,
       1.93869442, 1.23612328, 0.86531715, 0.52876874, 1.51939673,
       0.57074055, 0.41730802, 0.35504548, 0.28138168, 1.38660252,
       0.35251676, 0.79301576, 1.95017141, 0.91666688, 0.9062726 ,
       0.22752642, 0.37519547, 1.97475726, 0.50559697, 1.08369551,
       0.31446003, 1.50361925, 0.40531495, 0.96017612, 1.38637958,
       0.80653345, 2.25288171, 1.85812837, 0.49026359, 0.90971231,
       1.03725282, 0.40583411, 1.05515064, 0.41439919, 2.15436937,
       0.41574333, 0.87274467, 1.37799866, 0.46403439, 1.05191175,
       2.26770575, 0.65234565, 0.50261219, 0.33171039, 1.25470

In [23]:
# We save everything
eci_exp = eci_exp_new.copy()
eci_imp = eci_imp_new.copy()
pci = pci_new.copy()
eci_exp_out = pd.DataFrame()
eci_exp_out["exporter"], eci_exp_out[f"ann_{variant}"] = mcp_exp_rca.index, eci_exp
eci_exp_out.to_csv(f"annihilv1_complexity_exp_{variant}.csv", index = False, sep = "\t")
eci_imp_out = pd.DataFrame()
eci_imp_out["exporter"], eci_imp_out[f"ann_{variant}"] = mcp_imp_rca.index, eci_imp
eci_imp_out.to_csv(f"annihilv1_complexity_imp_{variant}.csv", index = False, sep = "\t")
pci_out = pd.DataFrame()
pci_out["prod"], pci_out[f"ann_{variant}"] = prod_order, pci
pci_out.to_csv(f"annihilv1_complexity_product_{variant}.csv", index = False, sep = "\t")